# Import statements

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from numpy import load, savez
import nets

# Folders for saving results

In [ ]:
# Path of the folder containing this jupiter notebook
folder = '/Users/admin/Documents/GitHub/Parametric-inverse-problem-topology/'

results_folder = folder + 'results/'
if not(os.path.exists(results_folder)):
        os.mkdir(results_folder)

results_folder = folder + 'results/Simple/'
if not(os.path.exists(results_folder)):
        os.mkdir(results_folder)
    
results_folder_direct = results_folder + 'Direct/'
if not(os.path.exists(results_folder_direct)):
        os.mkdir(results_folder_direct)
        
weights_folder_direct = results_folder_direct + 'weights/'
if not(os.path.exists(weights_folder_direct)):
        os.mkdir(weights_folder_direct)

results_folder_embedd = folder + 'results/Simple/Embedding/'
if not(os.path.exists(results_folder_embedd)):
        os.mkdir(results_folder_embedd)
        
weights_folder_embedd = results_folder_embedd + 'weights/'
if not(os.path.exists(weights_folder_embedd)):
        os.mkdir(weights_folder_embedd)

# Neural networks
Multilayer perceptrons composed by 4 hidden layers with 3000 neurons each. The activation function is the ReLU

In [ ]:
dimensions = [60, 3000, 3000, 3000, 3000, 1]

device = "cpu"        
model_direct = nets.net_loop_simple_direct(dimensions).to(device)
model_embedd = nets.net_loop_simple_embedd(dimensions).to(device)

# Datasets

In [ ]:
data = load(folder + 'data/dataset_loop_simple.npz')

# Visibilities training and validation
vis_train = data['vis_train']
max_vis_train = np.max(vis_train, axis = 1)
mmax_vis_train = np.expand_dims(max_vis_train, axis = 1)
mmax_vis_train = np.repeat(mmax_vis_train, 60, axis=1)
vis_train = vis_train / mmax_vis_train
vis_train = torch.from_numpy(vis_train)
vis_train = vis_train.to(device)

vis_valid = data['vis_valid']
max_vis_valid = np.max(vis_valid, axis = 1)
mmax_vis_valid = np.expand_dims(max_vis_valid, axis = 1)
mmax_vis_valid = np.repeat(mmax_vis_valid, 60, axis=1)
vis_valid = vis_valid / mmax_vis_valid
vis_valid = torch.from_numpy(vis_valid)
vis_valid = vis_valid.to(device)

# Labels direct approach
alpha_train = data['alpha_train']/180.
alpha_train = np.expand_dims(alpha_train, 1)
c_train = data['c_train']
c_train = np.expand_dims(c_train, 1)

target_train_direct = np.concatenate((alpha_train, c_train), axis=1 )
target_train_direct = torch.from_numpy(target_train_direct)
target_train_direct = target_train_direct.to(device)


alpha_valid = data['alpha_valid']/180.
alpha_valid = np.expand_dims(alpha_valid, 1)
c_valid = data['c_valid']
c_valid = np.expand_dims(c_valid, 1)

target_valid_direct = np.concatenate((alpha_valid, c_valid), axis=1 )
target_valid_direct = torch.from_numpy(target_valid_direct)
target_valid_direct = target_valid_direct.to(device)

# Labels approach with embedding
alpha_train = data['alpha_train']/180. * np.pi
c_train = data['c_train']

xx_train = (1. + c_train * np.sin(alpha_train)) * np.cos(2.*alpha_train)
xx_train = np.expand_dims(xx_train, 1) 
yy_train = (1. + c_train * np.sin(alpha_train)) * np.sin(2.*alpha_train)
yy_train = np.expand_dims(yy_train, 1) 
zz_train = np.cos(alpha_train) * c_train
zz_train = np.expand_dims(zz_train, 1) 

target_train_embedd = np.concatenate((xx_train, yy_train, zz_train), axis=1 )
target_train_embedd = torch.from_numpy(target_train_embedd)
target_train_embedd = target_train_embedd.to(device)

alpha_valid = data['alpha_valid']/180. * np.pi
c_valid = data['c_valid']

xx_valid = (1. + c_valid * np.sin(alpha_valid)) * np.cos(2.*alpha_valid)
xx_valid = np.expand_dims(xx_valid, 1) 
yy_valid = (1. + c_valid * np.sin(alpha_valid)) * np.sin(2.*alpha_valid)
yy_valid = np.expand_dims(yy_valid, 1) 
zz_valid = np.cos(alpha_valid) * c_valid
zz_valid = np.expand_dims(zz_valid, 1)

target_valid_embedd = np.concatenate((xx_valid, yy_valid, zz_valid), axis=1 )
target_valid_embedd = torch.from_numpy(target_valid_embedd)
target_valid_embedd = target_valid_embedd.to(device)

# Training direct approach

In [ ]:
num_epochs_train = 1000
batch_size       = 100
learning_rate    = 0.000001

#scheduler parameters
step_size = 100 
gamma     = 0.5
#number of batches in an epoch
n_batch   = int(vis_train.shape[0] / batch_size)

train_dataset = nets.VisDatasetTrain(vis_train.float(), target_train_direct.float())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# Loss function: Mean Squared Error
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_direct.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

#initialization of the arrays containing the values of the loss on training and validation set
train_loss = np.zeros(num_epochs_train)
valid_loss = np.zeros(num_epochs_train)

print('Train on {} data'.format(vis_train.shape[0]))
print('Validate on {} data'.format(vis_valid.shape[0]))
# Train, validation...
for epoch in range(num_epochs_train):
    model_direct.train()
    loss_epoch = np.zeros(n_batch)
    running_loss = 0
    for i, (vis, labels) in enumerate(train_loader):

        # clean the gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model_direct(vis)
        
        loss = criterion(outputs, labels)

        # backpropagation
        loss.backward()

        # optimise
        optimizer.step()

        # update the loss for visualisation/print
        loss_epoch[i] = loss.item()
        running_loss += loss.item()

    train_loss[epoch] = np.mean(loss_epoch)
    
    # for each epoch, compute also validation loss
    model_direct.eval()

    outputs = model_direct(vis_valid.float())
    loss = criterion(outputs, target_valid_direct.float())
    valid_loss[epoch] = loss.item()

    print('Epoch [{}/{}], Train loss: {:.7f}, Valid loss: {:.7f}'
          .format(epoch + 1, num_epochs_train, train_loss[epoch], valid_loss[epoch]))

    if (epoch+1) % 100 == 0:
        torch.save(model_direct.state_dict(), weights_folder_direct + '/nn_direct_' + str(epoch + 1) + '.pt')

    scheduler.step()

savez(results_folder_direct + '/history', train_loss = train_loss, valid_loss = valid_loss)

# Training approach with embedding

In [ ]:
num_epochs_train = 100
batch_size       = 100
learning_rate    = 0.000001

#scheduler parameters
step_size = 100 
gamma     = 0.5
#number of batches in an epoch
n_batch   = int(vis_train.shape[0] / batch_size)

train_dataset = nets.VisDatasetTrain(vis_train.float(), target_train_embedd.float())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# Loss function: Mean Squared Error
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_embedd.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

#initialization of the arrays containing the values of the loss on training and validation set
train_loss = np.zeros(num_epochs_train)
valid_loss = np.zeros(num_epochs_train)


print('Train on {} data'.format(vis_train.shape[0]))
print('Validate on {} data'.format(vis_valid.shape[0]))
# Train, validation...
for epoch in range(num_epochs_train):
    model_direct.train()
    loss_epoch = np.zeros(n_batch)
    running_loss = 0
    for i, (vis, labels) in enumerate(train_loader):

        # clean the gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model_embedd(vis)
        
        loss = criterion(outputs, labels)

        # backpropagation
        loss.backward()

        # optimise
        optimizer.step()

        # update the loss for visualisation/print
        loss_epoch[i] = loss.item()
        running_loss += loss.item()

    train_loss[epoch] = np.mean(loss_epoch)
    
    # for each epoch, compute also validation loss
    model_embedd.eval()

    outputs = model_embedd(vis_valid.float())
    loss = criterion(outputs, target_valid_embedd.float())
    valid_loss[epoch] = loss.item()

    print('Epoch [{}/{}], Train loss: {:.7f}, Valid loss: {:.7f}'
          .format(epoch + 1, num_epochs_train, train_loss[epoch], valid_loss[epoch]))

    if (epoch+1) % 25 == 0:
        torch.save(model_embedd.state_dict(), weights_folder_embedd + '/nn_embedd_' + str(epoch + 1) + '.pt')

    scheduler.step()

savez(results_folder_embedd + '/history', train_loss = train_loss, valid_loss = valid_loss)